In [1]:
# set cuda visible devices
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

import os
if is_notebook():
    os.environ["CUDA_VISIBLE_DEVICES"] = "" #"1"
    # os.environ['CUDA_LAUNCH_BLOCKING']="1"
    # os.environ['TORCH_USE_CUDA_DSA'] = "1"

import matplotlib 
if not is_notebook():
    matplotlib.use('Agg')

# set directory
os.chdir("/nas/ucb/oliveradk/diverse-gen/")

In [2]:
# for now incomplete spurious correlation: 
#   waterbirds

# other notebooks:
# multi-class classification: 
#   multi-nli-cc

# known group labels
# waterbirds (normal)
# mulit-nli cc 


In [3]:
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from itertools import product
import json

import numpy as np

from losses.loss_types import LossType
from utils.exp_utils import get_executor, run_experiments

In [4]:
SCRIPT_NAME = "measurement_tampering.py"
EXP_DIR = "output/mtd_hparam_sweep"
n_trials = 32

In [5]:
# diamonds hparams: 
## full model: 2e-5 
## probe: 2e-4 
## ground truth probes: 2e-4 
## num epochs: 5 
## num warmup stesp: 64 

# generated stories hparams: 
## full model: 1e-6
## probes: 5e-4 
## ground truth probes: 5e-3 
## num epochs: 4 
## num warmup stesp: 8

In [14]:
N_EPOCHS = 2

method_configs = {
    "TopK 0.1": {"loss_type": LossType.TOPK},
    "DivDis": {"loss_type": LossType.DIVDIS},
}

# TODO: set frac warmup steps appropriately for each dataset
dimaond_env_configs = {
    f"diamonds-seed{i}": {
        "dataset": f"diamonds-seed{i}", 
        "model": f"codegen-350M-mono-measurement_pred-diamonds-seed{i}", 
        "bootstrap_eval": False, 
        "frac_warmup_steps": 0.10, # 64 step
    } 
    for i in range(8)
}
dataset_configs = {
   **dimaond_env_configs,
   "generated-stories": {
        "dataset": "generated_stories", 
        "model": "pythia-1_4b-deduped-measurement_pred-generated_stories", 
        "bootstrap_eval": True, 
        "micro_batch_size": 2, 
        "max_length": 1536, 
        "feature_dim": 2048, 
        "frac_warmup_steps": 0.15, # 8 steps
   }
}

dataset_method_hparam_ranges = {
    **{f"diamonds-seed{i}": {
        "TopK 0.1": {"lr": [2e-5, 1e-3], "aux_weight": [1e-0, 3e-1]},
        "DivDis": {"lr": [2e-5, 1e-3], "aux_weight": [1e-0, 1e-2]},
        } for i in range(8)
    },
    "generated-stories": {
        "TopK 0.1": {"lr": [1e-6, 1e-3], "aux_weight": [1e-0, 1e-2]},
        "DivDis": {"lr": [1e-6, 1e-3], "aux_weight": [1e-0, 1e-2]},
    }
}

def sample_hparams(dataset_name, method_name):
    method_ranges = dataset_method_hparam_ranges[dataset_name][method_name]
    samples_hparams = {}
    for param_name, param_range in method_ranges.items():
        samples_hparams[param_name] = 10**(np.random.uniform(param_range[0], param_range[1]))
    return samples_hparams


In [16]:
configs = {(ds_name, method_name): {**ds_config, **method_config}
           for (ds_name, ds_config), (method_name, method_config) in product(dataset_configs.items(), method_configs.items())}

sampled_configs = []
for (ds_name, method_name), conf in configs.items():
    n_trials_ds = n_trials if not ds_name.startswith("diamonds") else n_trials // 8
    for i in range(n_trials_ds):
        samples_hparams = sample_hparams(ds_name, method_name)
        conf.update(samples_hparams)
        seed = np.random.randint(0, 1000000)
        conf["seed"] = seed
        conf["num_epochs"] = N_EPOCHS
        sampled_configs.append(conf)

def get_conf_exp_dir(ds_name, method_name, aux_weight, lr):
    return Path(EXP_DIR, f"{ds_name}_{method_name}/{aux_weight}-{lr}")

for (ds_name, method_name), conf in configs.items():
    exp_dir = get_conf_exp_dir(ds_name, method_name, conf["aux_weight"], conf["lr"])
    conf["exp_dir"] = exp_dir

In [ ]:
non_80gb_nodes = ["ddpg", "dqn", "gail", "gan","ppo", "vae"]
slurm_exclude = [f"{node}.ist.berkeley.edu" for node in non_80gb_nodes]

executor = get_executor(exp_dir, mem_gb=32, slurm_exclude=slurm_exclude)
jobs = run_experiments(sampled_configs, executor, SCRIPT_NAME)